### Programa para separar las observaciones en sub-integraciones temporales

Las observaciones deben estar dentro de un directorio llamado './pfds/'

Importamos los paquetes que vamos a usar

In [ ]:
import pypulse as pulsar
import matplotlib.pyplot as plt     # para graficar

import numpy as np
import glob                         # para hacer listas de archivo
import subprocess                   # para usar subprocesos

import os
import shutil

In [19]:
ant = "A2"     # antena usada (A1 o A2)
year = "2020"  # año de las observaciones
t_min = 2000   # duración mínima de las subintegraciones

Creamos una lista con todos los archivos .pfd de observaciones

In [ ]:
os.chdir('./pfds/')                         # entramos al directorio que contiene los .pfds
pfd_files = glob.glob('*' + ant + '_' + year + '*pfd')                           # lista de archivos .pfd

#print(pfd_files)
print("Numero de observaciones = " + str(len(pfd_files)))

output = "./results_" + ant + '_' + year + '_subints_' + str(t_min) + '.txt'     # archivo que contendrá S/N y t_obs
list = open(output, "w+")        

Procesamos las observaciones usando PyPulse

In [ ]:
# Convert PFD files to PSRFITS

for pfd in pfd_files:
    subprocess.check_call(['psrconv','-o','PSRFITS','-e','fits',pfd])
    
# Save all PSRFITS files
psrfits_files = glob.glob('*' + ant + '_' + year + '*fits')

# Load all PSRFITS into PyPulse
fits_pypulse= []
for psrfits in psrfits_files:
    temp_pypulse= pulsar.Archive(psrfits)
    fits_pypulse.append(temp_pypulse)

observations = dict(zip(psrfits_files, fits_pypulse))

# Create single pulse object
# Creat array of best profiles

single_pulses=[]

for observation in fits_pypulse:
    
    if observation.getDuration() > t_min:
    
    # First crunch in time and frequency
    
        n_subint = int((observation.getDuration()) // t_min) # número de subintegraciones para esta observación

        observation.fscrunch()
        observation.tscrunch(nsubint=n_subint)               # separamos en subintegraciones
    
#       print("Número de subintegraciones = " + str(observation.getNsubint()))
#       print("Duración de la observación total = " + str(observation.getDuration()/60))
#       print("Duración de cada subintegración = " + str(observation.getDurations()/60))
    
    # Now get array of the best profile
    
        tmp_singlepulse = observation.getSinglePulses( windowsize = int(observation.getNbin()/4 ))
        
        if n_subint == 1:   # en caso de que no haya subintegraciones, debe tratarse a la observación como única
        
            # Align and normalize
            tmp_singlepulse.center_align()
            tmp_singlepulse.normalize()
        
            file = str(observation).replace(".fits", ".pfd")
            t_obs = observation.getDuration()/60                          # calculamos el tiempo de integración en minutos
            sn_obs = tmp_singlepulse.getSN()                              # calculamos el S/N de la observación
 
            list.write(file + "   " + str(sn_obs) + "   " + str(t_obs) + "\n")  # y escribimos el S/N y el t_obs en el archivo de salida
    
        # Save aligned and normalize in each observation
            single_pulses.append(tmp_singlepulse)
            
        
        elif n_subint > 1:  # en caso de que sí haya subintegraciones, debe analizarse cada una por separado
    
            for n in range(observation.getNsubint()):  # barremos todas las subintegraciones
            
            # Align and normalize
                tmp_singlepulse[n].center_align()
                tmp_singlepulse[n].normalize()
    
#        print("S/N de la " + str(n) + "-esima subintegración = " + str(tmp_singlepulse[n].getSN()))
        
                file = str(observation).replace(".fits", ".pfd")
                t_obs = observation.getDurations()[n] / 60               # calculamos el tiempo de integración en minutos
                sn_obs = tmp_singlepulse[n].getSN()                      # calculamos el S/N de la observación
 
                list.write(file + "   " + str(sn_obs) + "   " + str(t_obs) + "\n")  # y escribimos el S/N y el t_obs en el archivo de salida
            
                # Save aligned and normalize in each observation
                single_pulses.append(tmp_singlepulse[n])
    
sp_observations = dict(zip(psrfits_files, single_pulses))

list.close()                               # cerramos el archivo de salida

os.chdir('..')                             # salimos de directorio que contiene los .pfds

shutil.move("./pfds/" + output, './')      # movemos el archivo con los resultados al directorio raíz

Una vez que tenemos los datos para las observaciones de todos los años, podemos juntarlos en un único archivo

In [ ]:
filenames = glob.glob("results_" + ant + '*' + '_subints_' + str(t_min) + '.txt')
with open("./results_" + ant + "_subints_" + str(t_min) + ".txt", 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)
